In [ ]:
import warnings
import pickle
import os

import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.exceptions import ConvergenceWarning
from sklearn.neural_network import MLPClassifier

from GeneticAlgorithm import GeneticAlgorithm as GA

In [ ]:

path_x = "storage/train-test-x.pkl"
path_y = "storage/train-test-y.pkl"
if not os.path.isfile(path_x):
    # Load data from https://www.openml.org/d/554
    X, y = fetch_openml('mnist_784', version=1, return_X_y=True)
    X = X / 255.

    # rescale the data, use the traditional train/test split
    X_train, X_test = X[:60000], X[60000:]
    y_train, y_test = y[:60000], y[60000:]

    file = open(path_x, 'wb')
    pickle.dump((X_train, X_test), file)
    file.close()
    
    file = open(path_y, 'wb')
    pickle.dump((y_train, y_test), file)
    file.close()
    
else:
    file = open(path_x, 'rb')# open a file, where you stored the pickled data
    X_train, X_test = pickle.load(file)
    file.close()

    file = open(path_y, 'rb')# open a file, where you stored the pickled data
    y_train, y_test = pickle.load(file)
    file.close()

In [ ]:
_ga = GA()

@_ga.fitness
def _fitness(individu):
    model = MLPClassifier(hidden_layer_sizes=tuple(individu["kromosom"]), max_iter=10, alpha=1e-4,
                        solver='sgd', verbose=10, random_state=1,
                        learning_rate_init=.1)

    # this example won't converge because of CI's time constraints, so we catch the warning and are ignore it here
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=ConvergenceWarning, module="sklearn")
        model.fit(X_train, y_train)
    individu["fitness"] = model.score(X_test, y_test)

In [ ]:
_ga.setFirstGeneration(restore=True)
_ga.population

In [ ]:
_ga.newGeneration()

In [ ]:
_ga.exportToCSV()

In [ ]:
_ga.saveGeneration()